In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import snappy
from sklearn.feature_selection import RFE, chi2, SelectKBest
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.metrics import confusion_matrix,roc_auc_score,classification_report,roc_curve
from mlxtend.feature_selection import ExhaustiveFeatureSelector

In [2]:
mask_file = r'\water_veg_mask'
mask_path = r'F:\workspace\jupyterNotebooks\Remote Sensing\ISRO\data'

In [3]:
water_veg_mask = np.load(r'.\data\water_veg_mask.npy')

In [4]:
unique, counts = np.unique(water_veg_mask, return_counts=True)
print(unique, counts)
water_veg_mask = water_veg_mask.flatten()

[0 1 2] [505368  14724 136008]


In [7]:
product_path = r'D:\SNAP\POSTINGARSS\final_S2A_MSIL2A_20190104T051211_N0211_R019_T43PGQ_20190104T094623_resampled.dim'

In [8]:
product = snappy.ProductIO.readProduct(product_path)
width = product.getSceneRasterWidth()
height = product.getSceneRasterHeight()

In [ ]:
B2 = product.getBand('B2')
B3 = product.getBand('B3')
B4 = product.getBand('B4')
B5 = product.getBand('B5')
B6 = product.getBand('B6')
B7 = product.getBand('B7')
B8 = product.getBand('B8')
B8A = product.getBand('B8A')
B11 = product.getBand('B11')
B12 = product.getBand('B12')

In [ ]:
B2_pixels = np.zeros(width * height, np.float32)
B2.readPixels(0,0,width,height,B2_pixels)
# B2_pixels.flatten()

B3_pixels = np.zeros(width * height, np.float32)
B3.readPixels(0,0,width,height,B3_pixels)
# B3_pixels.flatten()

B4_pixels = np.zeros(width * height, np.float32)
B4.readPixels(0,0,width,height,B4_pixels)
# B4_pixels.flatten()

B5_pixels = np.zeros(width * height, np.float32)
B5.readPixels(0,0,width,height,B5_pixels)

B6_pixels = np.zeros(width * height, np.float32)
B6.readPixels(0,0,width,height,B6_pixels)

B7_pixels = np.zeros(width * height, np.float32)
B7.readPixels(0,0,width,height,B7_pixels)

B8_pixels = np.zeros(width * height, np.float32)
B8.readPixels(0,0,width,height,B8_pixels)
# B8_pixels.flatten()

B8A_pixels = np.zeros(width * height, np.float32)
B8A.readPixels(0,0,width,height,B8A_pixels)

B11_pixels = np.zeros(width * height, np.float32)
B11.readPixels(0,0,width,height,B11_pixels)

B12_pixels = np.zeros(width * height, np.float32)
B12.readPixels(0,0,width,height,B12_pixels)

In [ ]:
df = pd.DataFrame({'B2':B2_pixels, 
                   'B3': B3_pixels,
                   'B4': B4_pixels,
                   'B5': B5_pixels,
                   'B6': B6_pixels,
                   'B7': B7_pixels,
                   'B8': B8_pixels,
                   'B8A': B8A_pixels,
                   'B11': B11_pixels,
                   'B12': B12_pixels,
                    'Y':water_veg_mask})

In [ ]:
df.head()

In [ ]:
plt.subplots(figsize=(10,10))
sns.heatmap(df.corr(method='kendall'), annot=True, cmap='coolwarm')

In [ ]:
sample_0 = df.query('Y == 0').sample(counts[1])
sample_1 = df.query('Y == 1').sample(counts[1])
sample_2 = df.query('Y == 2').sample(counts[1])

In [ ]:
sampled_df = pd.concat([sample_0, sample_1, sample_2],keys=['B2', 'B3', 'B4','B5','B6','B7', 'B8','B8A','B11','B12', 'Y'])
sampled_df = sampled_df.sample(frac=1).reset_index(drop=True)
sampled_df.describe()

In [ ]:
plt.subplots(figsize=(10,10))
sns.heatmap(sampled_df.corr(method='kendall'), annot=True, cmap='coolwarm')

In [ ]:
x_cols = ['B2','B3','B4','B5','B6','B7', 'B8','B8A','B11','B12']

In [ ]:
X = df[x_cols]
y = df['Y']

In [ ]:
# X_train,X_test_val,y_train,y_test_val = train_test_split(sampled_df[x_cols],sampled_df['Y'],test_size=0.4,random_state=1234, stratify = sampled_df['Y'])

In [ ]:
# X_test,X_val, y_test, y_val = train_test_split(X_test_val,y_test_val,test_size=0.5,random_state=4321, stratify = y_test_val)

In [ ]:
logreg = LogisticRegression()
svm = LinearSVC()
etc = ExtraTreesClassifier()

In [ ]:
rfe = RFE(estimator=svm, n_features_to_select=1)
rfe = rfe.fit(X, y.values.ravel())
print(x_cols)
print(rfe.support_)
print(rfe.ranking_)

In [ ]:
rfe = RFE(estimator=logreg, n_features_to_select=1)
rfe = rfe.fit(X, y.values.ravel())
print(x_cols)
print(rfe.support_)
print(rfe.ranking_)

In [ ]:
# unreliable
etc.fit(X,y)
print(etc.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization
feat_importances = pd.Series(etc.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

In [ ]:
#apply SelectKBest class to extract top n best features
n =10
bestfeatures = SelectKBest(score_func=chi2, k=n)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(n,'Score'))  #print 10 best features

In [ ]:
pca = PCA(n_components=4)
# X = StandardScaler().fit_transform(X)
principalComponents = pca.fit_transform(X)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2','principal component 3','principal component 4'])
finalDf = pd.concat([principalDf, df[['Y']]], axis = 1)

In [ ]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = [0,1,2]
colors = ['r', 'b', 'g']
for target, color in zip(targets,colors):
    indicesToKeep = finalDf['Y'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid()

In [ ]:
pca.explained_variance_ratio_

In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');